In [2]:
!pip install efficientnet
import efficientnet.tfkeras as efn

'pip' is not recognized as an internal or external command,
operable program or batch file.


ModuleNotFoundError: No module named 'efficientnet'

In [6]:
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt
from os.path import join
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import re

print("Tensorflow version " + tf.__version__)

Tensorflow version 2.4.1


In [7]:
from keras import backend as K

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
# Detect hardware, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


# Set the data path

In [10]:
files_path = 'drive/MyDrive/Colab Notebooks/sheethal sheethal/final_deliverable'
files_path

'drive/MyDrive/Colab Notebooks/sheethal sheethal/final_deliverable'

In [14]:
!ls -al 'drive/MyDrive/Colab Notebooks/sheethal sheethal/final_deliverable/models/'

total 3318086
-rw------- 1 root root  813740488 Feb 15 06:58 effnetB7_correct_plus_incorrect.h5
-rw------- 1 root root 2583978712 Feb 14 15:45 EffNetB7_correct_plus_incorrect.h5


# Set Parameters

In [ ]:
def count_images(dir):
  files = tf.io.gfile.glob(dir + '/*.tfrec')
  return sum([int(re.search(r'-([0-9]*)\.', file).group(1)) for file in files])

In [ ]:
IMAGE_SIZE = [224, 224] # at this size, a GPU will run out of memory. Use the TPU

# number of time the model will be fitted
EPOCHS = 5

# number of images trained at a time
BATCH_SIZE = 8 * strategy.num_replicas_in_sync

# get the count of train, val and test images
NUM_TRAINING_IMAGES = count_images(files_path + '/data/train_tfrecs')
NUM_VALIDATION_IMAGES = count_images(files_path + '/data/val_tfrecs')
NUM_TEST_IMAGES = count_images(files_path + '/data/test_tfrecs')

# number of batch of images to be trained per epoch
TRAIN_STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE + 1
VAL_STEPS_PER_EPOCH = NUM_VALIDATION_IMAGES // BATCH_SIZE + 1

# number of classes in the dataset
NUM_CLASSES = 3

# class mapping
CLASS_MAPPING = {0: 'Correct', '1': 'Hand Error', 2: 'Numbering Error'}

### Distribution of Classes in Training data

In [ ]:
train_data = pd.read_csv(files_path + '/data/train_label.csv')
train_data['error_type'].value_counts(normalize=True)

correct      0.33724
numbering    0.33364
hand         0.32912
Name: error_type, dtype: float64

### Distribution of Classes in Validation data

In [ ]:
val_data = pd.read_csv(files_path + '/data/val_label.csv')
val_data['error_type'].value_counts(normalize=True)

correct      0.3348
hand         0.3344
numbering    0.3308
Name: error_type, dtype: float64

# Load the data

In [ ]:
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.cast(image, tf.float32) / 255.0  # convert image to floats in [0, 1] range
    image = tf.reshape(image, [*IMAGE_SIZE, 3]) # explicit size needed for TPU
    return image

def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "image_name": tf.io.FixedLenFeature([], tf.string),  # shape [] means single element
        'hour': tf.io.FixedLenFeature([], tf.int64),
        'minute': tf.io.FixedLenFeature([], tf.int64),
        'error_type': tf.io.FixedLenFeature([], tf.string)
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])

    labels = [0, 0, 0]

    if example['error_type'] == 'correct':
      labels[0] = 1
    elif example['error_type'] == 'hand':
      labels[1] = 1
    elif example['error_type'] == 'numbering':
      labels[2] = 1

    # label = 0
    # if example['error_type'] == 'correct':
    #   label = 1
    # elif example['error_type'] == 'hand':
    #   label = 2
    # elif example['error_type'] == 'numbering':
    #   label = 3
    
    # label = 1 if example['error_type'] == 'correct' else 0
    return image, labels # returns a dataset of (image, label) pairs

def read_unlabeled_tfrecord(example):
    UNLABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "image_name": tf.io.FixedLenFeature([], tf.string),  # shape [] means single element
        # class is missing, this competitions's challenge is to predict flower classes for the test dataset
    }
    example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    idnum = example['image_name']
    return image, idnum # returns a dataset of image(s)

def load_dataset(filenames, labeled=True, ordered=False):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # disregarding data order. Order does not matter since we will be shuffling the data anyway.

    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed

    dataset = tf.data.TFRecordDataset(filenames) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(read_labeled_tfrecord if labeled else read_unlabeled_tfrecord)
    # returns a dataset of (image, label) pairs if labeled=True or (image, id) pairs if labeled=False
    return dataset

def get_training_dataset():
    dataset = load_dataset(tf.io.gfile.glob(files_path + '/data/train_tfrecs/*.tfrec'), labeled=True)
    dataset = dataset.repeat(EPOCHS) # the training dataset must repeat for several epochs
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

def get_validation_dataset():
    dataset = load_dataset(tf.io.gfile.glob(files_path + '/data/val_tfrecs/*.tfrec'), labeled=True, ordered=True)
    dataset = dataset.repeat(EPOCHS) # the training dataset must repeat for several epochs
    dataset = dataset.batch(BATCH_SIZE)
    # dataset = dataset.cache()
    return dataset

def get_test_dataset(ordered=False, labeled=False):
    dataset = load_dataset(tf.io.gfile.glob(files_path + '/data/test_tfrecs/*.tfrec'), labeled=labeled, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

training_dataset = get_training_dataset()
validation_dataset = get_validation_dataset()
# test_dataset = get_test_dataset()

# Build the Model - with 3 Classes (Correct, Hand Error & Numbering Error)

In [ ]:
with strategy.scope():
    effnetB7_pretrained_model =  efn.EfficientNetB7(weights='imagilenet', include_top=False, pooling='avg', input_shape=[*IMAGE_SIZE, 3])
    effnetB7_pretrained_model.trainable = True # False = transfer learning, True = fine-tuning
    
    effnetB7 = tf.keras.Sequential([
        effnetB7_pretrained_model,
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
    ])
        
effnetB7.compile(
    optimizer='adam',
    loss = 'categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
set the learning rate for the model
K.set_value(effnetB7.optimizer.learning_rate, 0.001)

fit the model
historical = effnetB7.fit(
              training_dataset, 
              steps_per_epoch=TRAIN_STEPS_PER_EPOCH, 
              epochs=5, 
              validation_data=validation_dataset,
              validation_steps=VAL_STEPS_PER_EPOCH
            )

## Store the model

In [ ]:
# effnetB7.save(join(files_path, 'models/effnetB7_correct_plus_incorrect.h5')) #.replace(' ', '\ ')

# Load the model

In [ ]:
# load the saved model
model = tf.keras.models.load_model(join(files_path, 'models/effnetB7_correct_plus_incorrect.h5'))

### Accuracy on the valiadtion dataset

In [ ]:
# accuracy on the validation dataset
model.evaluate(validation_dataset)

6250/6250 [==============================] - 684s 108ms/step - loss: 0.1910 - accuracy: 0.9338


[0.19096213579177856, 0.9337999820709229]

In [ ]:
# classification report
x_val = next(iter(validation_dataset.map(lambda img, label: img).unbatch().batch(NUM_VALIDATION_IMAGES)))
y_val = next(iter(validation_dataset.map(lambda img, label: label).unbatch().batch(NUM_VALIDATION_IMAGES))).numpy().argmax(axis=1)

y_val_pred = model.predict(x_val).argmax(axis=1)

print(classification_report(y_val, y_val_pred, target_names=list(CLASS_MAPPING.values())))

                 precision    recall  f1-score   support

        Correct       0.84      1.00      0.91      3348
     Hand Error       1.00      0.99      1.00      3344
Numbering Error       1.00      0.81      0.89      3308

       accuracy                           0.93     10000
      macro avg       0.94      0.93      0.93     10000
   weighted avg       0.94      0.93      0.93     10000



## Display Training curves

In [ ]:
def display_training_curves(training, validation, title, subplot):
    if subplot%10==1: # set up the subplots on the first call
        plt.subplots(figsize=(10,10), facecolor='#F0F0F0')
        plt.tight_layout()
    ax = plt.subplot(subplot)
    ax.set_facecolor('#F8F8F8')
    ax.plot(training)
    ax.plot(validation)
    ax.set_title('model '+ title)
    ax.set_ylabel(title)
    #ax.set_ylim(0.28,1.05)
    ax.set_xlabel('epoch')
    ax.legend(['train', 'valid'])

In [ ]:
display_training_curves(historical.history['loss'], historical.history['val_loss'], 'loss', 211)
display_training_curves(historical.history['accuracy'], historical.history['val_accuracy'], 'accuracy', 212)

# Make Predictions

In [ ]:
# load test data
test_dataset = get_test_dataset(labeled=False, ordered=True)

# extract the images
x_test = test_dataset.map(lambda img, imgid: img)

# extract the image ids
img_ids_test = next(iter(test_dataset.map(lambda img, imgid: imgid).unbatch().batch(NUM_TEST_IMAGES))).numpy()

# make predictions
y_test_pred = model.predict(x_test).argmax(axis=1)

# create data frame
submission = pd.DataFrame()
submission['image_id'] = img_ids_test
submission['label'] = y_test_pred
submission.head(100)

,image_id,label
0,b'0.jpg',2
1,b'1.jpg',2
2,b'2.jpg',2
3,b'3.jpg',2
4,b'4.jpg',0
...,...,...
95,b'95.jpg',2
96,b'96.jpg',2
97,b'97.jpg',0
98,b'98.jpg',2
